In [160]:
%matplotlib inline
import pickle
import urllib
import time
import feedparser
import itertools
from copy import deepcopy
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from tqdm import tqdm

In [232]:
category = 'astro-ph'
entries = pickle.load(open(category + '_entries.pkl', 'rb'))
author_ind = pickle.load( open(category + '_author_ind.pkl', 'rb'))
train_adj_list = pickle.load(open(category + '_train_adj_list.pkl', 'rb'))
test_adj_list = pickle.load(open(category + '_test_adj_list.pkl', 'rb'))
num_authors = len(author_ind)
authors = range(num_authors)
pos_edges = set([(min(a1, a2), max(a1, a2)) for (a1, a2) in \
                 itertools.combinations(authors, 2)]) - set(train_adj_list)
pred_edges = set(test_adj_list) - set(train_adj_list)
dist = pickle.load(open(category + '_dist.pkl', 'rb'))

In [357]:
def gen_neighbors(train_adj_list, author_ind):
    neighbors = {}
    for a in author_ind.iterkeys():
        neighbors[author_ind[a]] = frozenset()

    for e in train_adj_list:
        neighbors[e[0]] = neighbors[e[0]].union(set([e[1]]))
        neighbors[e[1]] = neighbors[e[1]].union(set([e[0]]))
    return neighbors

def common_neighbors(pos_edges, neighbors, authors, k):
    ranking = []
    for a1, a2 in pos_edges:
        neighbor_count = len(neighbors[a1].intersection(neighbors[a2]))
        ranking.append((a1, a2, neighbor_count))
    ranking.sort(key=lambda p: -p[2])
    ranking = [(r[:2]) for r in ranking]
    ranked_edges = set(ranking[:k])
    return ranked_edges

def adamic_adar(pos_edges, neighbors, k):
    ranking = []
    for e in tqdm(pos_edges):
        edge_neighbors = neighbors[e[0]].intersection(neighbors[e[1]])
        score = np.sum([(1.0/np.log(len(neighbors[n]) + 1e-10)) for n in edge_neighbors])
        ranking.append((e[0], e[1], score))
    ranking.sort(key=lambda p: -p[2])
    ranking = [(r[:2]) for r in ranking]
    ranked_edges = set(ranking[:k])
    return ranked_edges

def random_edges(pos_edges, k):
    ind = np.random.choice(range(len(pos_edges)), k, replace=False)
    pos_edges_list = list(pos_edges)
    pred_edges = set([pos_edges_list[i] for i in ind])
    return pred_edges

# Shortest distances
def floyd_warshall(train_adj_list, num_authors):
    dist = np.zeros((num_authors, num_authors)) + 1e10
    for e in train_adj_list:
        dist[e[0]][e[1]] = 1
        dist[e[1]][e[0]] = 1
    # Because symmetrical, only want to calculate dist[i][j] where j > i
    # This means that j > k, j > i, k > i.
    for k in tqdm(range(num_authors)):
        for i in range(k):
            for j in range(k + 1, num_authors):
                if dist[i][j] > dist[i][k] + dist[k][j]:
                    dist[i][j] = dist[i][k] + dist[k][j]
    return dist

def build_matrix(train_adj_list, author_ind):
    P = np.zeros((len(author_ind), len(author_ind))) + 1e-50
    for e in train_adj_list:
        P[e[0]][e[1]] = 1
        P[e[1]][e[0]] = 1        
    return P

# Rooted pagerank
def rooted_pagerank(P, alpha, e, threshold=1e-3):
    n = P.shape[0]
    P_norm = np.divide(P.T, (P.sum(axis=1)))
    
    # Start at x
    s_o = np.zeros((n))
    s_i = np.array([0] * n)
    s_i[e[0]] = 1
    
    # Random prob alpha to jump to x
    E = np.array([0] * n)
    E[e[0]] = alpha
        
    while (np.sum(np.abs(s_i - s_o)) > threshold):
        s_o = s_i
        s_i = np.dot(((1 - alpha) * P_norm + E), s_o)
    
    return s_i[e[1]]

# Generate matrix with rooted pagerank for all possible edges 
def generate_pr_mat(P, edges, alpha):
    num_authors = P.shape[0]
    root_pr = {}
    for e in tqdm(edges):
        root_pr[e] = rooted_pagerank(P, alpha, e)
    return root_pr

def graph_distance(dist, num_authors, k):
    # Shortest distance edges
    shortest_dist = []
    for i in range(num_authors):
        for j in range(i + 1, num_authors):
            if dist[i][j] == 2:
                shortest_dist.append((i, j))
    # Generally, distance of 2 will be completely sufficient
    ind = np.random.choice(range(len(shortest_dist)), k, replace=False)    
    pred_edges = set([shortest_dist[i] for i in ind])
    return pred_edges, shortest_dist

def pred_acc(pred_edges, ranked_edges):
    tot_edges = float(len(pred_edges))
    cor_edges = pred_edges.intersection(ranked_edges)
    return len(cor_edges) / tot_edges

In [346]:
neighbors = gen_neighbors(train_adj_list, author_ind)
P = build_matrix(train_adj_list, author_ind)
cn_edges = common_neighbors(pos_edges, neighbors, authors, len(pred_edges))
rand_edges = random_edges(pos_edges, len(pred_edges))
closest_edges, all_closest = graph_distance(dist, num_authors, len(pred_edges))

In [358]:
aa_edges = adamic_adar(pos_edges, neighbors, len(pred_edges))

100%|██████████| 3258886/3258886 [00:21<00:00, 155060.48it/s]


In [ ]:
# Need to parallelize this, currently takes ~15 hours
pr_edges = generate_pr_mat(P, all_closest, 0.15)

In [360]:
# Top K where K is number of test set edges
print "Random edges accuracy: %0.4f" % pred_acc(rand_edges, pred_edges)
print "Common neighbors accuracy: %0.4f" % pred_acc(cn_edges, pred_edges)
print "Graph distance accuracy: %0.4f" % pred_acc(closest_edges, pred_edges)
print "Adamic adar accuracy: %0.4f" % pred_acc(aa_edges, pred_edges)

Random edges accuracy: 0.0024
Common neighbors accuracy: 0.0923
Graph distance accuracy: 0.0333
Adamic adar accuracy: 0.1020
